# Optimal Portland Ballpark Location

The goal of this project is to determine the optimal location for a hypothetical Major League Baseball Stadium in Portland, Oregon, based on similarity to other urban ballpark neighborhoods and access to public transportation. 

### Setup
Import and install necessary packages and libraries

In [731]:
import pandas as pd
!conda install -c conda-forge folium=0.5.0 --yes
import folium
import requests
from pandas.io.json import json_normalize
import numpy as np
from sklearn.cluster import KMeans
from geopy.geocoders import Nominatim
import matplotlib.cm as cm
import matplotlib.colors as colors
import geopy.distance
from sklearn import preprocessing

Solving environment: done

# All requested packages already installed.



### Create Ballpark Locations

Create dataframe of existing urban ballparks and their locations. Data are entered manually since no existing public source was found. 


Wrigley Field, Chicago = Chc

Fenway Park, Boston = Bos

Coors Field, Denver = Col

Nationals Park, Washington D.C. = Was

Oriole Park at Camden Yards, Baltimore = Bal

PNC Park, Pittsburgh = Pit

Yankee Stadium, New York = Nyy

T-Mobile Park, Seattle = Sea

Petco Park, San Diego = Sd

Minute Maid Park, Houston = Hou

Busch Stadium, St. Louis = Stl

Chase Field, Phoenix = Arz

In [955]:
locs = [['Chc',41.948437,-87.655334], ['Bos',42.346268,-71.095764],['Col',39.7559,-104.9942],['Was',38.8730,-77.0074],['Bal',39.2839,-76.6216],['Pit',40.4469,-80.0057],['Nyy',40.8296,-73.9262],['Sea',47.5914,-122.3325],['Sd',32.7076,-117.1570],['Hou',29.7573,-95.3555],['Stl',38.6226,-90.1928],['Arz',33.4453,-112.0667]]
dfl=pd.DataFrame(locs, columns = ['Stadium','Lat','Long'])
dfl

,Stadium,Lat,Long
0,Chc,41.948437,-87.655334
1,Bos,42.346268,-71.095764
2,Col,39.755900,-104.994200
3,Was,38.873000,-77.007400
4,Bal,39.283900,-76.621600
5,Pit,40.446900,-80.005700
6,Nyy,40.829600,-73.926200
7,Sea,47.591400,-122.332500
8,Sd,32.707600,-117.157000
9,Hou,29.757300,-95.355500


## Nearby Venues Function

Define a function which uses calls to foursquare API for location/venue data to obtain a list of venues nearby a given location. A 600 meter radius is used based on trial-and-error to sufficiently distinguish neighborhoods and return meaningful results. 

In [956]:
def getNearbyVenues(names, latitudes, longitudes, radius=600):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Stadium', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

User information for Foursquare calls

In [835]:
CLIENT_ID = 'N5XIY3IIQOWL05E0BKAC31DYDP12D54INLICRBEOZIU3IBJ3'
CLIENT_SECRET = 'DOLZ305KGS5Q2ZE2MQJRNL53MJQW3CVCHIMZPD0RNPHD3GZX'
VERSION = '20180605'
LIMIT=100

Make the call to the Nearby Venues function to create a data frame of venues within the radius of each existing urban ballpark

In [957]:
venues = getNearbyVenues(names=dfl['Stadium'],latitudes=dfl['Lat'],longitudes=dfl['Long'])

Chc
Bos
Col
Was
Bal
Pit
Nyy
Sea
Sd
Hou
Stl
Arz


In [959]:
##Uncomment this block to check the head of the Venues table if desired
#venues.head(30)

Determine the number of venues within the radius of each stadium

In [960]:
venues.groupby('Stadium').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Stadium,,,,,,
Arz,55,55,55,55,55,55
Bal,54,54,54,54,54,54
Bos,100,100,100,100,100,100
Chc,91,91,91,91,91,91
Col,94,94,94,94,94,94
Hou,39,39,39,39,39,39
Nyy,64,64,64,64,64,64
Pit,71,71,71,71,71,71
Sd,100,100,100,100,100,100


Implement one-hot encoding to begin venue frequency calculation

In [961]:
# one hot encoding
onehot = pd.get_dummies(venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
onehot['Stadium'] = venues['Stadium'] 

# move neighborhood column to the first column
fixed_columns = [onehot.columns[-1]] + list(onehot.columns[:-1])
onehot = onehot[fixed_columns]

onehot.head()

,Yoga Studio,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,...,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the one-hot encoding by stadium to determine the frequency of each venue type occurrence near each stadium

In [962]:
grouped = onehot.groupby('Stadium').mean().reset_index()
grouped

,Stadium,Yoga Studio,ATM,Accessories Store,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Train Station,Vegetarian / Vegan Restaurant,Video Store,Vietnamese Restaurant,Volleyball Court,Whisky Bar,Wine Bar,Wine Shop,Winery,Wings Joint
0,Arz,0.000000,0.000000,0.000000,0.072727,0.018182,0.018182,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.018182,0.00,0.000000,0.000000
1,Bal,0.000000,0.000000,0.000000,0.018519,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
2,Bos,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.010000,0.000000,0.01,0.000000,0.000000,0.010000,0.00,0.000000,0.000000
3,Chc,0.000000,0.000000,0.000000,0.010989,0.010989,0.010989,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.010989,0.00,0.000000,0.010989,0.000000,0.00,0.000000,0.000000
4,Col,0.010638,0.000000,0.000000,0.063830,0.000000,0.010638,0.010638,0.000000,0.000000,...,0.010638,0.000000,0.000000,0.00,0.000000,0.010638,0.000000,0.00,0.000000,0.000000
5,Hou,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
6,Nyy,0.000000,0.000000,0.000000,0.015625,0.000000,0.000000,0.000000,0.000000,0.015625,...,0.015625,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000
7,Pit,0.000000,0.000000,0.000000,0.042254,0.000000,0.000000,0.000000,0.000000,0.014085,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.014085
8,Sd,0.000000,0.000000,0.000000,0.050000,0.000000,0.000000,0.000000,0.010000,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.010000,0.000000,0.01,0.000000,0.000000
9,Sea,0.000000,0.016949,0.016949,0.016949,0.000000,0.000000,0.000000,0.016949,0.000000,...,0.000000,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00,0.000000,0.000000


Sum the columns to determine the stereotypical characterization of an urban ballpark "locale"

In [963]:
grouped.loc['Column_Total']= grouped.sum(numeric_only=True, axis=0)
locale = grouped.T.sort_values(by='Column_Total',ascending=False).drop('Baseball Stadium')
locale = locale['Column_Total']
locale

Bar                         0.616576
Hotel                       0.603777
Sports Bar                   0.50998
American Restaurant         0.454324
Pizza Place                 0.385708
Coffee Shop                 0.363345
Mexican Restaurant          0.348757
Sandwich Place              0.274306
Park                        0.260207
Italian Restaurant          0.217984
Burger Joint                0.213547
Lounge                      0.207142
Steakhouse                  0.201657
Beer Garden                 0.199178
Café                        0.154976
General Entertainment        0.15319
Restaurant                  0.144084
Food Truck                  0.139918
Seafood Restaurant          0.132139
Clothing Store              0.128828
Gym                         0.127254
Sporting Goods Shop         0.123904
Brewery                     0.123357
Cocktail Bar                0.121999
BBQ Joint                   0.118114
Baseball Field                0.1084
Outdoor Sculpture           0.108319
F

Check top 10 venue frequencies around each ballpark. 
Note: This is not required for the code to function, merely as a check and to provide additional insight.

In [964]:
num_top_venues = 10

for park in grouped['Stadium']:
    print("----"+park+"----")
    temp = grouped[grouped['Stadium'] == park].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Arz----
                 venue  freq
0  American Restaurant  0.07
1          Pizza Place  0.07
2                  Bar  0.07
3               Lounge  0.07
4          Coffee Shop  0.05
5   Basketball Stadium  0.05
6           Sports Bar  0.04
7   Mexican Restaurant  0.04
8                  Gym  0.02
9     Storage Facility  0.02


----Bal----
               venue  freq
0     Sandwich Place  0.11
1              Hotel  0.07
2  Outdoor Sculpture  0.07
3                Pub  0.06
4                Gym  0.04
5   Baseball Stadium  0.04
6        Coffee Shop  0.04
7          Hotel Bar  0.04
8          Gastropub  0.02
9         Steakhouse  0.02


----Bos----
                 venue  freq
0          Coffee Shop  0.05
1  American Restaurant  0.05
2           Sports Bar  0.05
3          Pizza Place  0.05
4                 Café  0.04
5         Burger Joint  0.03
6  Japanese Restaurant  0.03
7           Restaurant  0.03
8      Thai Restaurant  0.03
9                Hotel  0.03


----Chc----
           

TypeError: must be str, not float

## Begin assessment of Portland Neighbhourhoods
Read in Portland Zip Code/Latitude & Longitude Data

In [967]:
url='https://github.com/dsknourek/Data_Course/blob/master/us-zip-code-latitude-and-longitude.csv'

zips = pd.read_html(url)
zips=zips[0]
zips=zips.drop(columns=['Unnamed: 0'])
zips.head()

,Zip,Latitude,Longitude
0,97298,45.548616,-123.114725
1,97233,45.513242,-122.499850
2,97223,45.438840,-122.782240
3,97281,45.548616,-123.114725
4,97238,45.580557,-122.374776


Create a dataframe of the venues nearby each zip code in Portland

In [968]:
pdx_venues = getNearbyVenues(names=zips['Zip'],latitudes=zips['Latitude'],longitudes=zips['Longitude'])

97298
97233
97223
97281
97238
97215
97207
97296
97293
97267
97299
97213
97266
97209
97256
97217
97206
97231
97230
97229
97254
97202
97222
97258
97232
97227
97259
97294
97271
97228
97292
97225
97283
97205
97236
97219
97286
97255
97216
97203
97210
97214
97208
97240
97239
97221
97204
97291
97220
97272
97224
97218
97242
97251
97290
97268
97201
97211
97280
97282
97212
97269
97253


Check the head of the dataframe for confirmation/insight

In [970]:
pdx_venues = pdx_venues.rename(columns={"Stadium":"Zip"})
pdx_venues.head()

,Zip,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,97233,45.513242,-122.49985,Hello Cupcake,45.513060,-122.497200,Cupcake Shop
1,97233,45.513242,-122.49985,Black Diamond Studio,45.512496,-122.496063,Music Venue
2,97233,45.513242,-122.49985,Parklane Park,45.513492,-122.503873,Playground
3,97223,45.438840,-122.78224,Rose City Futsal West,45.440952,-122.781476,Soccer Field
4,97223,45.438840,-122.78224,Paradise Harley-Davidson,45.442273,-122.781285,Motorcycle Shop


Check the counts of venues near each zip code

In [971]:
counts=pdx_venues.groupby('Zip').count().reset_index()
counts

,Zip,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,97201,10,10,10,10,10,10
1,97202,8,8,8,8,8,8
2,97203,27,27,27,27,27,27
3,97204,100,100,100,100,100,100
4,97205,100,100,100,100,100,100
5,97206,2,2,2,2,2,2
6,97207,6,6,6,6,6,6
7,97208,25,25,25,25,25,25
8,97209,100,100,100,100,100,100
9,97210,19,19,19,19,19,19


Define "busy" neighborhoods as those with at least 20 venues nearby

In [995]:
busy = counts[counts['Venue Category']>19]
busy

,Zip,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
2,97203,27,27,27,27,27,27
3,97204,100,100,100,100,100,100
4,97205,100,100,100,100,100,100
7,97208,25,25,25,25,25,25
8,97209,100,100,100,100,100,100
13,97214,47,47,47,47,47,47
15,97216,37,37,37,37,37,37
16,97217,34,34,34,34,34,34
24,97227,23,23,23,23,23,23
28,97232,38,38,38,38,38,38


Create a dataframe of just the busy-area zip codes

In [996]:
busy_zips = busy['Zip'].reset_index().drop(columns='index')
busy_zips

,Zip
0,97203
1,97204
2,97205
3,97208
4,97209
5,97214
6,97216
7,97217
8,97227
9,97232


Trim the venues dataframe to only include the busy neighborhoods

In [997]:
pdx_venues = pdx_venues[pdx_venues['Zip'].isin(busy_zips['Zip'])]
pdx_venues.head()

,Zip,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
109,97266,45.478641,-122.56233,Foster Feed And Garden,45.477107,-122.557081,Pet Store
110,97266,45.478641,-122.56233,Planet Fitness,45.479359,-122.568422,Gym / Fitness Center
111,97266,45.478641,-122.56233,Zoiglhaus,45.480967,-122.568522,Brewery
112,97266,45.478641,-122.56233,Redbox,45.478337,-122.563289,Video Store
113,97266,45.478641,-122.56233,7-Eleven,45.478051,-122.563277,Convenience Store


Implement one-hot encoding for the Portland venues

In [998]:
# one hot encoding
pdx_onehot = pd.get_dummies(pdx_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
pdx_onehot['Zip'] = pdx_venues['Zip'] 

# move neighborhood column to the first column
fixed_columns = [pdx_onehot.columns[-1]] + list(pdx_onehot.columns[:-1])
pdx_onehot = pdx_onehot[fixed_columns]

pdx_onehot.head()

,Zip,ATM,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
109,97266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110,97266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
111,97266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
112,97266,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
113,97266,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Group the one-hot dataframe by zip code

In [999]:
pdx_grouped = pdx_onehot.groupby('Zip').mean().reset_index()
pdx_grouped

,Zip,ATM,American Restaurant,Antique Shop,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Automotive Shop,...,Thrift / Vintage Store,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Weight Loss Center,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,97203,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.037037,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
1,97204,0.000000,0.020000,0.000000,0.000000,0.00,0.00,0.03,0.000000,0.00,...,0.000000,0.020000,0.00,0.000000,0.020000,0.00,0.010000,0.010000,0.000000,0.010000
2,97205,0.000000,0.030000,0.000000,0.000000,0.01,0.01,0.00,0.010000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.010000,0.010000,0.000000
3,97208,0.000000,0.040000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.04,0.000000,0.000000,0.04,0.000000,0.000000,0.000000,0.000000
4,97209,0.000000,0.030000,0.010000,0.010000,0.00,0.01,0.00,0.010000,0.01,...,0.000000,0.020000,0.00,0.000000,0.020000,0.00,0.010000,0.010000,0.010000,0.020000
5,97214,0.000000,0.021277,0.021277,0.021277,0.00,0.00,0.00,0.000000,0.00,...,0.042553,0.021277,0.00,0.000000,0.000000,0.00,0.021277,0.000000,0.000000,0.000000
6,97216,0.027027,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.027027,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.000000
7,97217,0.029412,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.058824,0.000000,0.00,0.000000,0.000000,0.029412,0.000000
8,97227,0.000000,0.000000,0.000000,0.000000,0.00,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.043478
9,97232,0.000000,0.000000,0.026316,0.000000,0.00,0.00,0.00,0.000000,0.00,...,0.000000,0.000000,0.00,0.000000,0.026316,0.00,0.000000,0.026316,0.000000,0.000000


Combine the Portland neighborhood venue frequencies with the associated venue frequencies of the "typical" urban ballparks. Then, calculate the n-dimensional Euclidean distance between the frequencies of each venue in the given zip code and in the typical ballpark neighborhood. Then, within each zipcode, take the square root of the sum of the squares of these distances to determine a similarity score. The lower the similarity score, the less distance between the venues in the given neighborhood and the typical ballpark neighborhood, and the better the location in the context of this project. Print out the top 10 most frequent venues in each neighborhood and the similarity score of the neighborhood.

In [1000]:
num_top_venues = 10
scores=pd.DataFrame(columns=["Score"])

for code in pdx_grouped['Zip']:
    print("----"+str(code)+"----")
    temp = pdx_grouped[pdx_grouped['Zip'] == code].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp=temp.join(locale, how='right', on='venue')
    temp = temp.round({'freq': 2}).fillna(value=0)
    temp['dist_sq']= (temp['freq']-temp['Column_Total'])**2
    dist= np.sqrt(temp['dist_sq'].sum())
    scores.loc[code]=[dist]
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print("Distance = "+str(dist.round(2)))
    print('\n')

----97203----
                 venue  freq  Column_Total   dist_sq
0          Coffee Shop  0.07      0.363345  0.086051
1                 Bank  0.07      0.052254  0.000315
2    Convenience Store  0.07      0.029157  0.001668
3           Food Truck  0.07      0.139918  0.004889
4                  Bar  0.07      0.616576  0.298745
5       Discount Store  0.04      0.018519  0.000461
6  Rental Car Location  0.04      0.057297  0.000299
7                 Food  0.04      0.010638  0.000862
8              Brewery  0.04      0.123357  0.006948
9   Mexican Restaurant  0.04      0.348757  0.095331
Distance = 1.49


----97204----
                venue  freq  Column_Total   dist_sq
0               Hotel  0.11      0.603777  0.243816
1         Coffee Shop  0.09      0.363345  0.074717
2          Food Truck  0.04      0.139918  0.009984
3          Restaurant  0.04      0.144084  0.010834
4    Asian Restaurant  0.03      0.020000  0.000100
5      Sandwich Place  0.03      0.274306  0.059685
6      

Check the similarity scores of the zip codes

In [1001]:
scores

,Score
97203,1.485495
97204,1.452330
97205,1.476603
97208,1.510405
97209,1.476939
97214,1.468847
97216,1.495155
97217,1.499460
97227,1.455881
97232,1.475030


Normalize the scores to contextualize the numbers

In [1002]:
min_max_scaler = preprocessing.MinMaxScaler()
score_scaled = min_max_scaler.fit_transform(scores)
scores['Score']=score_scaled

scores.sort_values(by='Score')

,Score
97204,0.000000
97227,0.061152
97214,0.284403
97232,0.390879
97205,0.417970
97209,0.423740
97266,0.501798
97203,0.571084
97216,0.737416
97217,0.811542


Create a dataframe of the Portland rail station locations and print the head of the dataframe for confirmation

In [1003]:
url='https://github.com/dsknourek/Data_Course/blob/master/tm_rail_stops.csv'
rail_stops = pd.read_html(url)
rail_stops=rail_stops[0].drop(columns=['Unnamed: 0'])
rail_stops=rail_stops.drop(columns=['Name','description','station','line','status','type']).reset_index()
rail_stops.head()

,index,X,Y
0,0,-122.661934,45.516937
1,1,-122.691150,45.520409
2,2,-122.667837,45.530187
3,3,-122.682089,45.577109
4,4,-122.661839,45.527412


Add latitude and longitude data back into the Busy Zip Code dataframe

In [1004]:
busy_zips = busy_zips.join(zips.set_index('Zip'),how="left",on='Zip')
busy_zips

,Zip,Latitude,Longitude
0,97203,45.589689,-122.738750
1,97204,45.518540,-122.675500
2,97205,45.520540,-122.685730
3,97208,45.532165,-122.564824
4,97209,45.528890,-122.684580
5,97214,45.513790,-122.645800
6,97216,45.515674,-122.560870
7,97217,45.574439,-122.686650
8,97227,45.543831,-122.674970
9,97232,45.528740,-122.644600


Add to the Busy Zip Code data so that it has a row corresponding to each rail stop

In [1005]:
for stop in rail_stops.T:
    busy_zips[stop]=float("NaN")

busy_zips=busy_zips.set_index('Zip')
busy_zips['ZipCode']=busy_zips.index

For every zip code, check every station and see if it is within a kilometer of the zip code. 

In [1006]:
for hood in busy_zips['ZipCode']:
    curr = busy_zips[busy_zips['ZipCode']==hood]
    coord1= (float(curr['Latitude']),float(curr['Longitude']))
    for stop in rail_stops['index']:
        curr_stop = rail_stops[rail_stops['index']==stop]
        coord2 = (float(curr_stop['Y']), float(curr_stop['X']))
        busy_zips.loc[hood,stop]= geopy.distance.distance(coord1,coord2).km <1
  
        
  
busy_zips=busy_zips.drop(columns=['Latitude','Longitude','ZipCode'])
busy_zips.head()     

,0,1,2,3,4,5,6,7,8,9,...,164,165,166,167,168,169,170,171,172,173
Zip,,,,,,,,,,,,,,,,,,,,,
97203,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
97204,False,False,False,False,False,False,False,False,False,False,...,True,True,True,True,False,False,False,True,True,True
97205,False,True,False,False,False,False,False,False,False,False,...,True,True,False,True,False,False,False,False,True,True
97208,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
97209,False,False,False,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,True,False,False


Count the number of train stops within a kilometer of each zip code

In [1007]:
busy_zips['Count']= busy_zips.sum(axis=1)
busy_zips.sort_values(by='Count', ascending=False)

,0,1,2,3,4,5,6,7,8,9,...,165,166,167,168,169,170,171,172,173,Count
Zip,,,,,,,,,,,,,,,,,,,,,
97204,False,False,False,False,False,False,False,False,False,False,...,True,True,True,False,False,False,True,True,True,44
97205,False,True,False,False,False,False,False,False,False,False,...,True,False,True,False,False,False,False,True,True,41
97209,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,True,False,False,30
97216,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
97217,False,False,False,True,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
97227,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,2
97208,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
97232,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1
97266,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,1


Join the similarity score of each zip code with the number of train stops within a kilometer of the zip code

In [1008]:
scores.join(busy_zips['Count']).sort_values('Score')

,Score,Count
97204,0.000000,44
97227,0.061152,2
97214,0.284403,0
97232,0.390879,1
97205,0.417970,41
97209,0.423740,30
97266,0.501798,1
97203,0.571084,0
97216,0.737416,2
97217,0.811542,2
